<a href="https://colab.research.google.com/github/TatKhachatryan/Web-Scrapping-Begginer-Intermediate-Advanced-/blob/main/Advanced_level_Web_Scraping_(https_fbref_com_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Scraper | EPL** (https://fbref.com)

In [11]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import datetime
import itertools
import numpy as np
from bs4 import BeautifulSoup
import unicodedata
import requests
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

In [13]:
seasons = ['https://fbref.com/en/comps/9/Premier-League-Stats',
           'https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats',
           'https://fbref.com/en/comps/9/2020-2021/2020-2021-Premier-League-Stats',
           'https://fbref.com/en/comps/9/2019-2020/2019-2020-Premier-League-Stats', 
           'https://fbref.com/en/comps/9/2018-2019/2018-2019-Premier-League-Stats', 
           'https://fbref.com/en/comps/9/2017-2018/2017-2018-Premier-League-Stats', 
           'https://fbref.com/en/comps/9/2016-2017/2016-2017-Premier-League-Stats', 
           'https://fbref.com/en/comps/9/2015-2016/2015-2016-Premier-League-Stats', 
           'https://fbref.com/en/comps/9/2014-2015/2014-2015-Premier-League-Stats', 
           'https://fbref.com/en/comps/9/2013-2014/2013-2014-Premier-League-Stats', 
           'https://fbref.com/en/comps/9/2012-2013/2012-2013-Premier-League-Stats', 
           'https://fbref.com/en/comps/9/2011-2012/2011-2012-Premier-League-Stats', 
           'https://fbref.com/en/comps/9/2010-2011/2010-2011-Premier-League-Stats', 
           'https://fbref.com/en/comps/9/2009-2010/2009-2010-Premier-League-Stats', 
           'https://fbref.com/en/comps/9/2008-2009/2008-2009-Premier-League-Stats', 
           'https://fbref.com/en/comps/9/2007-2008/2007-2008-Premier-League-Stats', 
           'https://fbref.com/en/comps/9/2006-2007/2006-2007-Premier-League-Stats', 
           'https://fbref.com/en/comps/9/2005-2006/2005-2006-Premier-League-Stats']


In [14]:
def scrapeTable(lst, index=0, season='22/23'):
    df = pd.read_html(lst[index])

    tables = []
    for idx,table in enumerate(df):
        tables.append(table)
    table = tables[0][0::1]
    table = table[['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts']]
    table.columns = ['Position', 'Club', 'Played', 'Won', 'Drawn', 'Lost', 'GF', 'GA', 'GD', 'Points']
    table['Season'] = season
    return table

In [15]:
def scrapePasses(lst, index=0, season='22/23'):
    """
    'xAG': Expected Assisted Goals,
    'xA': Expected Assists,
    'A-xAG': Assists-Expected Assisted Goals,
    'KP': Passes that directly lead to a shot (assisted shots,
    '1/3': Completed  passes that enter the 1/3 of the pitch closest to the goal,
    'PPA': Completed  passes into the 18-yard box,
    'CrsPA': Completed crosses into the 18-yard box,
    'Prog': Progressive passes
    """
        
    df = pd.read_html(lst[index])

    tables = []
    for idx,table in enumerate(df):
        tables.append(table)
    table = tables[10]

    table.columns = ['Team', 'Num of Players in game', 'Minutes Played / 90 minutes', 'TotPassCompleted', 'TotPassAttended', 
             'TotPassCompleted_%', 'TotDistance', 'ProgDist_%', 'ShortPassCompleted', 'ShortPassAttended', 'ShortPassCompleted_%', 
             'MediumPassCompleted', 'MediumPassAttended', 'MediumPassCompleted_%', 'LongPassCompleted', 'LongPassAttended',
             'LongPassCompleted_%', 'Assists', 'xAG', 'xA', 'A-xAG', 'KP', '1/3', 'PPA', 'CrsPA', 'ProgPasses']
    table['Season'] = season
    return table

In [16]:
def scrapeShooting(lst, index=0, season='22/23'):
    df = pd.read_html(lst[index])
    tables = []
    for idx,table in enumerate(df):
        tables.append(table)
        
    if index <= 5:
        table = tables[8]
        table.columns = table.columns.droplevel()
    elif index > 5:
        table = tables[6]
        table.columns = table.columns.droplevel()
    table = table[['Squad', '# Pl', '90s', 'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'PK', 'PKatt']]
    table.columns = ['Team', 'Num of Players in game', 'Minutes Played / 90 minutes', 'Goals', 'TotalShots', 'ShotsOnTarget', 
             'ShotsOnTarget_%', 'Shots_per90', 'ShotsOnTarget_per90', 'Goals_perShot', 'Goals_perShotOnTarget', 
             'AvgDistance', 'PenaltyMade', 'PenaltyAttended']
    table['Season'] = season
    return table

In [17]:
def scrapeClubStats(lst, index=0, season='22/23'):
    df = pd.read_html(lst[index])

    tables = []
    for idx,table in enumerate(df):
        tables.append(table)
    table = tables[22]
    table.columns = ['Team', 'Num of Players in game', 'Minutes Played / 90 minutes', 'YellowCard', 'RedCard', 'SecondYellowCard', 'FoulsCommited',
            'Fouls Drawn', 'Offsides', 'Crosses', 'Interceptions', 'TklW', 'PenaltyWon', 'PenaltyConceded', 'OwnGoal', 'BallsRecov', 'AerialWon', 'AerialLost', 'Won%']

    table[['Num of Players in game', 'Minutes Played / 90 minutes', 'YellowCard', 'RedCard', 'SecondYellowCard', 'FoulsCommited', 'Fouls Drawn', 'Offsides', 'Crosses', 'Interceptions',  'tackle_successCount', 'PenaltyWon', 'PenaltyConceded', 'OwnGoal', 'BallsRecov', 'AerialWon', 'AerialLost', 'Won%']] = table[['Num of Players in game', 'Minutes Played / 90 minutes', 'YellowCard', 'RedCard', 'SecondYellowCard', 'FoulsCommited', 'Fouls Drawn', 'Offsides', 'Crosses', 'Interceptions', 'TklW', 'PenaltyWon', 'PenaltyConceded', 'OwnGoal', 'BallsRecov', 'AerialWon', 'AerialLost', 'Won%']].astype(float)

    table['AerialBattles'] = table['AerialWon']+table['AerialLost']
    table['Season'] = season
    return table

In [18]:
se2023table = scrapeTable(seasons, 0, '22/23')
se2022table = scrapeTable(seasons, 1, '21/22')
se2021table = scrapeTable(seasons, 2, '20/21')
se2020table = scrapeTable(seasons, 3, '19/20')
se2019table = scrapeTable(seasons, 4, '18/19')
se2018table = scrapeTable(seasons, 5, '17/18')
se2017table = scrapeTable(seasons, 6, '16/17')
se2016table = scrapeTable(seasons, 7, '15/16')
se2015table = scrapeTable(seasons, 8, '14/15')
se2014table = scrapeTable(seasons, 9, '13/14')
se2013table = scrapeTable(seasons, 10, '12/13')
se2012table = scrapeTable(seasons, 11, '11/12')
se2011table = scrapeTable(seasons, 12, '10/11')
se2010table = scrapeTable(seasons, 13, '09/10')
se2009table = scrapeTable(seasons, 14, '08/09')
se2008table = scrapeTable(seasons, 15, '07/08')
se2007table = scrapeTable(seasons, 16, '06/07')
se2006table = scrapeTable(seasons, 17, '05/06')
frames = [se2023table, se2022table, se2021table, se2020table, se2019table, se2018table, se2017table, se2016table, se2015table, 
          se2014table, se2013table, se2012table, se2011table, se2010table, se2009table, se2008table, se2007table, se2006table]

tables = pd.concat(frames)
# tables.to_csv('tables.csv', index=False)
tables.head()

,Position,Club,Played,Won,Drawn,Lost,GF,GA,GD,Points,Season
0,1,Arsenal,13,11,1,1,31,11,20,34,22/23
1,2,Manchester City,13,10,2,1,39,12,27,32,22/23
2,3,Newcastle Utd,14,7,6,1,28,11,17,27,22/23
3,4,Tottenham,14,8,2,4,27,18,9,26,22/23
4,5,Manchester Utd,13,7,2,4,18,19,-1,23,22/23


In [19]:
# commented variables mean that there were no data found for corresponding season

se2023Passes = scrapePasses(seasons, 0, '22/23')
se2022Passes = scrapePasses(seasons, 1, '21/22')
se2021Passes = scrapePasses(seasons, 2, '20/21')
se2020Passes = scrapePasses(seasons, 3, '19/20')
se2019Passes = scrapePasses(seasons, 4, '18/19')
se2018Passes = scrapePasses(seasons, 5, '17/18')
# se2017Passes = scrapePasses(seasons, 6, '16/17')
# se2016Passes = scrapePasses(seasons, 7, '15/16')
# se2015Passes = scrapePasses(seasons, 8, '14/15')
# se2014Passes = scrapePasses(seasons, 9, '13/14')
# se2013Passes = scrapePasses(seasons, 10, '12/13')
# se2012Passes = scrapePasses(seasons, 11, '11/12')
# se2011Passes = scrapePasses(seasons, 12, '10/11')
# se2010Passes = scrapePasses(seasons, 13, '09/10')
# se2009Passes = scrapePasses(seasons, 14, '08/09')
# se2008Passes = scrapePasses(seasons, 15, '07/08')
# se2007Passes = scrapePasses(seasons, 16, '06/07')
# se2006Passes = scrapePasses(seasons, 17, '05/06')

frames = [se2023Passes, se2022Passes, se2021Passes, se2020Passes, se2019Passes, se2018Passes]

passes = pd.concat(frames)
# passes.to_csv('passes.csv', index=False)
passes.head()

,Team,Num of Players in game,Minutes Played / 90 minutes,TotPassCompleted,TotPassAttended,TotPassCompleted_%,TotDistance,ProgDist_%,ShortPassCompleted,ShortPassAttended,ShortPassCompleted_%,MediumPassCompleted,MediumPassAttended,MediumPassCompleted_%,LongPassCompleted,LongPassAttended,LongPassCompleted_%,Assists,xAG,xA,A-xAG,KP,1/3,PPA,CrsPA,ProgPasses,Season
0,Arsenal,23,13.0,6002,7207,83.3,103958,35640,2767,3059,90.5,2600,2931,88.7,507,882,57.5,23,16.9,14.3,6.1,155,521,144,23,456,22/23
1,Aston Villa,23,14.0,5066,6501,77.9,89985,33170,2271,2600,87.3,2128,2509,84.8,527,1027,51.3,8,11.1,10.2,-3.1,109,403,94,31,343,22/23
2,Bournemouth,25,14.0,4176,5496,76.0,74911,30456,1873,2174,86.2,1739,2069,84.1,440,895,49.2,12,7.4,8.4,4.6,84,282,74,23,234,22/23
3,Brentford,24,14.0,4473,5984,74.7,81519,32398,2066,2419,85.4,1761,2168,81.2,534,1060,50.4,11,11.7,10.2,-0.7,101,341,99,26,296,22/23
4,Brighton,21,13.0,5662,7012,80.7,99528,34293,2569,2885,89.0,2389,2758,86.6,554,991,55.9,11,15.4,14.4,-4.4,148,473,132,28,410,22/23


In [21]:
se2023Shoots = scrapeShooting(seasons, 0, '22/23')
se2022Shoots = scrapeShooting(seasons, 1, '21/22')
se2021Shoots = scrapeShooting(seasons, 2, '20/21')
se2020Shoots = scrapeShooting(seasons, 3, '19/20')
se2019Shoots = scrapeShooting(seasons, 4, '18/19')
se2018Shoots = scrapeShooting(seasons, 5, '17/18')
se2017Shoots = scrapeShooting(seasons, 6, '16/17')
se2016Shoots = scrapeShooting(seasons, 7, '15/16')
se2015Shoots = scrapeShooting(seasons, 8, '14/15')
se2014Shoots = scrapeShooting(seasons, 9, '13/14')
se2013Shoots = scrapeShooting(seasons, 10, '12/13')
se2012Shoots = scrapeShooting(seasons, 11, '11/12')
se2011Shoots = scrapeShooting(seasons, 12, '10/11')
se2010Shoots = scrapeShooting(seasons, 13, '09/10')
se2009Shoots = scrapeShooting(seasons, 14, '08/09')
se2008Shoots = scrapeShooting(seasons, 15, '07/08')
se2007Shoots = scrapeShooting(seasons, 16, '06/07')
se2006Shoots = scrapeShooting(seasons, 17, '05/06')

frames = [se2023Shoots, se2022Shoots, se2021Shoots, se2020Shoots, se2019Shoots, se2018Shoots, se2017Shoots, 
          se2016Shoots, se2015Shoots, se2014Shoots, se2013Shoots, se2012Shoots, se2011Shoots, se2010Shoots, 
          se2009Shoots, se2008Shoots, se2007Shoots, se2006Shoots]

shoots = pd.concat(frames)
# shoots.to_csv('shoots.csv', index=False)
shoots.head()

In [22]:
# commented variables mean that there were no data found for corresponding season

se2023ClubStats = scrapeClubStats(seasons, 0, '22/23')
se2022ClubStats = scrapeClubStats(seasons, 1, '21/22')
se2021ClubStats = scrapeClubStats(seasons, 2, '20/21')
se2020ClubStats = scrapeClubStats(seasons, 3, '19/20')
se2019ClubStats = scrapeClubStats(seasons, 4, '18/19')
se2018ClubStats = scrapeClubStats(seasons, 5, '17/18')
# se2017ClubStats = scrapeClubStats(seasons, 6, '16/17')
# se2016ClubStats = scrapeClubStats(seasons, 7, '15/16')
# se2015ClubStats = scrapeClubStats(seasons, 8, '14/15')
# se2014ClubStats = scrapeClubStats(seasons, 9, '13/14')
# se2013ClubStats = scrapeClubStats(seasons, 10, '12/13')
# se2012ClubStats = scrapeClubStats(seasons, 11, '11/12')
# se2011ClubStats = scrapeClubStats(seasons, 12, '10/11')
# se2010ClubStats = scrapeClubStats(seasons, 13, '09/10')
# se2009ClubStats = scrapeClubStats(seasons, 14, '08/09')
# se2008ClubStats = scrapeClubStats(seasons, 15, '07/08')
# se2007ClubStats = scrapeClubStats(seasons, 16, '06/07')
# se2006ClubStats = scrapeClubStats(seasons, 17, '05/06')

frames = [se2023ClubStats, se2022ClubStats, se2021ClubStats, se2020ClubStats, se2019ClubStats, se2018ClubStats]

clubStats = pd.concat(frames)
# clubStats.to_csv('clubStats.csv', index=False)
clubStats.head()